<a href="https://colab.research.google.com/github/JonathanSum/TorchAudio_and_TorchTextNotes/blob/main/Capture_patterns_with_recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

     |████████████████████████████████| 24.2 MB 1.7 MB/s 
     |████████████████████████████████| 1.4 MB 33.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 50.4 MB 19 kB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 1.9 MB 27.3 MB/s 
     |████████████████████████████████| 7.1 MB 39.5 MB/s 
     |████████████████████████████████| 17.4 MB 203 kB/s 
     |████████████████████████████████| 1.9 MB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.7 MB/s 
     |████████████████████████████████| 895 kB 38.9 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434689 sha256=32feebaf507bb0bbd9daf4c4f86b74342c97d66ba9c4c8819ca92c5b88853a55
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Unins

In [2]:
!wget -q https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/torchnlp.py

In [3]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...


train.csv: 29.5MB [00:00, 82.3MB/s]
test.csv: 1.86MB [00:00, 27.3MB/s]                  


Building vocab...


In [4]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.308125
6400: acc=0.38390625
9600: acc=0.46239583333333334
12800: acc=0.51453125
16000: acc=0.554375
19200: acc=0.5868229166666666
22400: acc=0.6121428571428571
25600: acc=0.63484375
28800: acc=0.6536111111111111
32000: acc=0.66903125
35200: acc=0.6814772727272728
38400: acc=0.6933333333333334
41600: acc=0.7032932692307692
44800: acc=0.7128125
48000: acc=0.7207708333333334
51200: acc=0.7282421875
54400: acc=0.7349080882352941
57600: acc=0.7407465277777778
60800: acc=0.7474177631578948
64000: acc=0.75178125
67200: acc=0.7563690476190477
70400: acc=0.7613068181818182
73600: acc=0.7660054347826087
76800: acc=0.7701953125
80000: acc=0.77325
83200: acc=0.7772836538461538
86400: acc=0.7810185185185186
89600: acc=0.7841517857142857
92800: acc=0.7873706896551724
96000: acc=0.79046875
99200: acc=0.7931552419354839
102400: acc=0.795859375
105600: acc=0.7985227272727272
108800: acc=0.801139705882353
112000: acc=0.8033303571428572
115200: acc=0.8056510416666667
118400: acc=0.80771114864

(0.03282213745117187, 0.8085833333333333)

In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

In [7]:
t1 = torch.tensor([[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]])

In [8]:
t1.shape

torch.Size([3, 5])

In [14]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [15]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)

3200: acc=0.288125
6400: acc=0.3459375
9600: acc=0.4060416666666667
12800: acc=0.454609375
16000: acc=0.49625
19200: acc=0.5338020833333333
22400: acc=0.5651339285714285
25600: acc=0.591328125
28800: acc=0.614375
32000: acc=0.634625
35200: acc=0.6518181818181819
38400: acc=0.6684635416666667
41600: acc=0.6822355769230769
44800: acc=0.6944642857142858
48000: acc=0.70475
51200: acc=0.71517578125
54400: acc=0.72375
57600: acc=0.7313020833333334
60800: acc=0.7389473684210527
64000: acc=0.745671875
67200: acc=0.7520238095238095
70400: acc=0.7577840909090909
73600: acc=0.7632472826086957
76800: acc=0.7682552083333334
80000: acc=0.7732125
83200: acc=0.7774158653846154
86400: acc=0.7815972222222223
89600: acc=0.7852678571428572
92800: acc=0.7890301724137931
96000: acc=0.79246875
99200: acc=0.7957762096774194
102400: acc=0.799169921875
105600: acc=0.8019886363636364
108800: acc=0.8045404411764706
112000: acc=0.8072946428571428
115200: acc=0.8098784722222222
118400: acc=0.8124831081081081


(0.02992859903971354, 0.8136)

In [12]:
class LSTMPackClassifier1(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        return self.fc(h[-1])

In [13]:
net1 = LSTMPackClassifier1(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net1,train_loader_len, lr=0.001,use_pack_sequence=True)

3200: acc=0.285
6400: acc=0.34578125
9600: acc=0.40270833333333333
12800: acc=0.458671875
16000: acc=0.50625
19200: acc=0.5458333333333333
22400: acc=0.5781696428571429
25600: acc=0.60625
28800: acc=0.6280555555555556
32000: acc=0.64803125
35200: acc=0.665
38400: acc=0.679765625
41600: acc=0.6927163461538461
44800: acc=0.7044196428571429
48000: acc=0.7151875
51200: acc=0.72427734375
54400: acc=0.7320036764705883
57600: acc=0.739375
60800: acc=0.7466776315789474
64000: acc=0.75315625
67200: acc=0.7592410714285714
70400: acc=0.7650710227272727
73600: acc=0.770366847826087
76800: acc=0.7747526041666667
80000: acc=0.7792875
83200: acc=0.7832932692307693
86400: acc=0.7874652777777778
89600: acc=0.7912165178571429
92800: acc=0.7949030172413794
96000: acc=0.7981770833333334
99200: acc=0.8014012096774193
102400: acc=0.804140625
105600: acc=0.8068276515151516
108800: acc=0.8093290441176471
112000: acc=0.8119375
115200: acc=0.81421875
118400: acc=0.8166638513513513


(0.02971452433268229, 0.81755)